In [ ]:
Для восстаноылкния информации и заполнения пропусков

In [1]:
import requests
import time
import json
import os
import datetime
import boto3
import pytz
from dateutil.relativedelta import relativedelta

In [1]:
from config import ACCESS_KEY,SECRET_KEY,TOKEN,FOLDER,FOLDER_ID,BUCKET_NAME,BUCKET_NAME

In [ ]:
TIME_ZONE = os.getenv("TIME_ZONE", "Europe/Moscow") #настройка функции
TEMP_FILENAME = "/tmp/temp_file"

headers={'Authorization':TOKEN ,'Accept':'application/json'}

def get_now_datetime_str(): # получаем актуальное время
    time_zone = os.getenv("TIME_ZONE", "Europe/Moscow") # меняем таймзону на московскую
    now = datetime.datetime.now(pytz.timezone(time_zone))    
    yesterday = now - datetime.timedelta(days=1) #нужна вчерашняя дата так как данные за прошлый день
    last_month_data = now - relativedelta(month=1)
    return {'key': yesterday.strftime('year=%Y/month=%m/%d.csv'),
            'key_month': yesterday.strftime('year=%Y/month=%m.csv'),
            'now':now.strftime('%Y-%m-%d %H:%M:%S'),
            'yesterday_data':yesterday.strftime('%Y-%m-%d'),
            'yesterday':yesterday.strftime('%Y-%m-%d %H:%M:%S'), 
            'year':yesterday.strftime('%Y'),
            'month':yesterday.strftime('%m'),
            'day':yesterday.strftime('%d'),
            'last_month_data':last_month_data.strftime('%Y-%m-%d')
            }

def create_query(): #функция создает новый запрос и возвращает id для запроса результата
    body = {
        "name":query_name, 
        "TYPE":"ANALYTICS", 
        "text":query_text, 
        "description":query_description
    }
    response = requests.post(
        f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries?project={FOLDER_ID}',
        headers=headers,
        json=body
    )
    if response.status_code == 200:
        return response.json()["id"]
    return f' Code: {response},  text: {response.text}'


def get_request(offset): # фунция возвращает ответ запроса. Максимум 1000 строк.
    offset = offset
    get_query_results_url = f'https://api.yandex-query.cloud.yandex.net/api/fq/v1/queries/{request_id}/results/0?project={FOLDER_ID}&offset={str(offset)}&limit=1000'
    response = requests.get(
        get_query_results_url,
        headers = headers
    )
    return response

def if_cell_is_list(cell): # функция участвует в преобразовании данных при создании файла
    if isinstance(cell, list):
        if len(cell) == 0:
            return ''
        else: 
            return cell[0]
    else:
        return cell

def write_temp_file():
    offset = 0
    response = get_request(offset) #запрашиваем данные запроса
    columns = [rows['name'] for rows in response.json()['columns']] #выделяем названия столбцов
    special_str = ""
    for j in columns:
        special_str = f"{special_str}{str(j)},"
    temp_file = open(TEMP_FILENAME, 'w')
    temp_file.write(special_str[:-1]+'\n')

    while response.status_code == 200 and len(response.json()['rows']) != 0:  #Цикл делает запросы по 10000, пока не кончатся данные
        response = get_request(offset)
        response_rows = response.json()['rows']
        rows = [[if_cell_is_list(cell) for cell in row] for row in response_rows]  #Преобразуются строки
        # Открывает созданный файл и добавляет в него строки
        for i in rows:
            special_str = ""
            for j in i:
                if isinstance(j, str):
                    special_str = f"{special_str}'{str(j).replace("'", "")}',"
                else:
                    special_str = f"{special_str}{str(j).replace("'", "")},"
            temp_file.write(special_str[:-1]+'\n') 
        offset +=1000 # увеличивает смещение

def get_s3_instance(): # функция создает соединение
    session = boto3.session.Session()
    return session.client(
        aws_access_key_id=ACCESS_KEY,
        aws_secret_access_key=SECRET_KEY,
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net'
    )

def upload_dump_to_s3(): # функция выгружает данные в s3
    get_s3_instance().upload_file(
        Filename=TEMP_FILENAME,
        Bucket=BUCKET_NAME,
        Key=key
    )

def remove_temp_files(): #функция удаляет временный файл
    os.remove(TEMP_FILENAME)

Ключи определяются по месяцам а не дням

In [19]:
import pandas as pd

start_date = datetime.datetime.strptime('2023-12-01','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2024-02-01','%Y-%m-%d').date()
dates_pd = pd.DataFrame({
        'date': pd.date_range(start=start_date, end=end_date,freq='MS'),
        'date_key': pd.date_range(start=start_date, end=end_date,freq='MS').strftime('year=%Y/month=%m.csv')
        })

dates_pd

,date,date_key
0,2023-12-01,year=2023/month=12.csv
1,2024-01-01,year=2024/month=01.csv
2,2024-02-01,year=2024/month=02.csv


In [ ]:
for i in range(0,dates_pd.shape[0]):
    key = f"{FOLDER}/{dates_pd.loc[i,['date_key']].values[0]}"
    date = f"{FOLDER}/{dates_pd.loc[i,['date']].values[0]}"
    now = get_now_datetime_str()['now']

    query_text = open('query.txt','r').read().format(date)
    query_name = f'{FOLDER} {now}' #имя, которое появляется в запросах
    query_description = f'запрос по восстановлению данных - {date}'#описание, которое появляется в запросах

    request_id = create_query()

    while str(get_request(0)) == '<Response [400]>':
        time.sleep(10)  # пауза для создания запроса

    write_temp_file()
    get_s3_instance()
    upload_dump_to_s3()
    remove_temp_files()